# Классификация статей

Импортируем нужные библиотеки

In [1]:
import numpy as np
import sklearn.feature_extraction.text, re
import sklearn.linear_model
import sklearn.metrics
import sklearn.grid_search
import pymorphy2
from sklearn.pipeline import Pipeline

Делаем паттерн для создания мешка слов для статей

In [2]:
WORD_PATTERN = '[а-я]+|[a-z]+'

Множества интересный и скучных статей из размеченной выборки.

In [3]:
interesting = set()
boring = set()
data = list()

Морфологический анализатор

In [4]:
morph = pymorphy2.MorphAnalyzer()

In [5]:
def get_text(article_path):
    with open(article_path, 'r') as html:
        page = html.read()
        words = re.findall(WORD_PATTERN, page)
        parsed_words = list()
        for word in words:
            parsed_word = morph.parse(word)[0]
            if parsed_word.tag.POS not in {'PREP', 'CONJ', 'PRCL', 'INTJ'}:
                parsed_words.append(word)
        page = ' '.join(parsed_words)
        return page


Загрузка размеченных статей

In [6]:
with open('interesting.txt', 'r') as lines:
    while True:
        article_name = lines.readline().rstrip()
        if article_name == '':
            break
        article = get_text(article_name)
        interesting.add(article)
        data.append(article)

with open('boring.txt', 'r') as lines:
    while True:
        article_name = lines.readline().rstrip()
        if article_name == '':
            break
        article = get_text(article_name)
        boring.add(article)
        data.append(article)

Пермешаем данных для лучшей рабаты классификатора

In [7]:
import random
random.shuffle(data)

Выделим из данных вектор фичей tf и idf

In [8]:
vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(token_pattern=WORD_PATTERN)
X = vectorizer.fit_transform(data)
Y = np.array([1 if t in interesting else 0 for t in data])

Создадим выоборки для обучение, поиска параметров и тестирования

In [9]:
border = 60
X_train, X_test = X[:border], X[border:]
Y_train, Y_test = Y[:border], Y[border:]

Обучение классификатора и оценка обучение

In [10]:
cls = sklearn.linear_model.SGDClassifier(loss='log')
cls.fit(X_train, Y_train)

metric = sklearn.metrics.roc_auc_score

Y_pred = cls.predict_proba(X_test)[:, 1]

score = metric(Y_test, Y_pred)
print('Score: ', score)

Score:  0.509539842873


## Кросс-валидация

Создадим pipeline для объединения работы vectorize и классификатора 

In [11]:
article_classifier = text_clf = Pipeline([('tfidf', sklearn.feature_extraction.text.TfidfVectorizer()),
                                          ('classifier', sklearn.linear_model.SGDClassifier(loss='log'))])

In [12]:
border = 60
X_train, X_test = data[:border], data[border:]
Y_train, Y_test = Y[:border], Y[border:]

Сетка парметров

In [13]:
grid = {
    'classifier__penalty': ['elasticnet'],
    'classifier__alpha': [0.001, 0.0001, 0.00001, 0.000001, 0.0000001],
    'classifier__l1_ratio': [0.0, 0.01, 0.05, 0.10, 0.2, 0.3, 0.4, 0.5],
    'tfidf__max_df': [0.85, 0.9, 0.95, 1.0],
    'tfidf__min_df': [0.01, 0.05, 0.10, 0.15],
}

Поиск параметров

In [14]:
def scorer(estimator, X, Y):
    return metric(Y, estimator.predict_proba(X)[:, 1])

In [15]:
searcher = sklearn.grid_search.GridSearchCV(                 
    estimator=article_classifier,
    param_grid=grid,                                         
    scoring=scorer,                                          
    cv=5,                                                    
    n_jobs=1                                                 
)                                                            

Оценка качества

In [16]:
searcher.fit(X_train, Y_train);      
print(searcher.best_score_)            
print(searcher.best_params_)           
                                       
best_cls = searcher.best_estimator_    
                                       
print(scorer(best_cls, X_test, Y_test))

0.771349206349
{'classifier__l1_ratio': 0.01, 'tfidf__max_df': 0.85, 'classifier__penalty': 'elasticnet', 'tfidf__min_df': 0.05, 'classifier__alpha': 1e-06}
0.585858585859


Можно видеть, что кросс-валидация улучшила работу классификатора